In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [ ]:
#之前的agg方法 不论是官方的'mean' 'std'等还是自己定义的 都是数值运算
#再看一些其他的分组运算

In [2]:
df = DataFrame({'key1':['a','a','b','b','a'],
                'key2':['one','two','one','two','one'],
                'data1':np.random.randn(5),
                'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,2.984187,0.342718
1,a,two,1.409260,-0.258133
2,b,one,0.552138,0.858217
3,b,two,-0.095376,0.342444
4,a,one,-0.862872,-1.166031


In [4]:
k1_mean = df.groupby('key1').mean().add_prefix('mean_')
k1_mean

,mean_data1,mean_data2
key1,,
a,1.176858,-0.360482
b,0.228381,0.600330


In [5]:
pd.merge(df,k1_mean,left_on='key1',right_index=True)

,key1,key2,data1,data2,mean_data1,mean_data2
0,a,one,2.984187,0.342718,1.176858,-0.360482
1,a,two,1.409260,-0.258133,1.176858,-0.360482
4,a,one,-0.862872,-1.166031,1.176858,-0.360482
2,b,one,0.552138,0.858217,0.228381,0.600330
3,b,two,-0.095376,0.342444,0.228381,0.600330


In [6]:
people = DataFrame(np.random.randn(5,5),
                   columns=['a','b','c','d','e'],
                   index=['Joe','Steve','Wes','Jim','Travis'])
people

,a,b,c,d,e
Joe,-1.081264,2.149122,-0.302495,-0.213970,-1.976803
Steve,0.764555,1.007407,-0.701412,0.812980,-1.052113
Wes,0.345863,-1.288100,-1.384223,-0.074422,-1.757063
Jim,2.234085,0.750185,-1.191997,0.400283,-1.108858
Travis,-1.214306,-0.314780,-1.208029,0.361796,0.771474


In [10]:
key = ['one','two','one','two','one']
people.groupby(key).min()
#要注意的就是此时参数只给了一个数组 意思就是重命名index后 按照index进行聚合

,a,b,c,d,e
one,-1.214306,-1.288100,-1.384223,-0.213970,-1.976803
two,0.764555,0.750185,-1.191997,0.400283,-1.108858


In [11]:
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,-0.649902,0.182081,-0.964916,0.024468,-0.987464
Steve,1.499320,0.878796,-0.946704,0.606632,-1.080485
Wes,-0.649902,0.182081,-0.964916,0.024468,-0.987464
Jim,1.499320,0.878796,-0.946704,0.606632,-1.080485
Travis,-0.649902,0.182081,-0.964916,0.024468,-0.987464


In [13]:
print(df.groupby(['key1']).mean())
df.groupby(['key1']).transform(np.mean)
#transform和直接使用的区别在于不会将用于groupby的key抽出来
#而是会保留原来的结构 将数据放在该放的位置（所以普遍出现多个位置相同数据的情况）

         data1     data2
key1                    
a     1.176858 -0.360482
b     0.228381  0.600330


,data1,data2
0,1.176858,-0.360482
1,1.176858,-0.360482
2,0.228381,0.600330
3,0.228381,0.600330
4,1.176858,-0.360482


In [14]:
def demean(arr):
    return arr-arr.mean()
demeaned = people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,-0.431361,1.967041,0.662421,-0.238438,-0.989339
Steve,-0.734765,0.128611,0.245292,0.206349,0.028373
Wes,0.995765,-1.470181,-0.419308,-0.098890,-0.769599
Jim,0.734765,-0.128611,-0.245292,-0.206349,-0.028373
Travis,-0.564403,-0.496860,-0.243113,0.337328,1.758938


In [16]:
demeaned.groupby(key).mean()

,a,b,c,d,e
one,0.000000e+00,7.401487e-17,-1.110223e-16,0.000000e+00,7.401487e-17
two,-1.110223e-16,0.000000e+00,0.000000e+00,5.551115e-17,1.110223e-16


In [ ]:
#agg和transform方法有严格的限制：传入的函数只能产生两种结果：1.产生一个可以广播的标量值（如np.mean）
#2.产生一个相同大小的结果数组
#apply方法则是最一般化的聚合方法

In [17]:
tips = pd.read_csv('../examples/tips.csv')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [18]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [24]:
def top(df,n=5,column='tip_pct'):
    # print(df.sort_values(by=column))
    return df.sort_values(by=column)[-n:]
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [26]:
print(tips.groupby('smoker').mean())
tips.groupby('smoker').apply(top)
#groupby之后就形成了两个部分 一个是Yes 一个是No
#top方法在DataFrame的各个片段上调用，即Yes和No上分别调用 然后将yes和no部分连接起来

        total_bill       tip      size   tip_pct
smoker                                          
No       19.188278  2.991854  2.668874  0.159328
Yes      20.756344  3.008710  2.408602  0.163196


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [29]:
tips.groupby('smoker').apply(top,n=1,column='total_bill')
#tips.groupby('smoker').apply(top(n=1,column='total_bill'))
#注意 注释的方法是错误的 因为如果显示调用的话 top的第一个参数是没有的
#如果按照非注释的写法 则第一个参数默认是前面的对象 后面是其他的参数
# 但是 我非常不喜欢这样的写法 非常非常不喜欢 不合理

,,total_bill,tip,smoker,day,time,size,tip_pct
smoker,,,,,,,,
No,212,48.33,9.0,No,Sat,Dinner,4,0.186220
Yes,170,50.81,10.0,Yes,Sat,Dinner,3,0.196812


In [31]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [36]:
result.T

smoker,No,Yes
count,151.000000,93.000000
mean,0.159328,0.163196
std,0.039910,0.085119
min,0.056797,0.035638
25%,0.136906,0.106771
50%,0.155625,0.153846
75%,0.185014,0.195059
max,0.291990,0.710345


In [37]:
#.describe方法实际使用就是使用apply方法
f = lambda x:x.describe()
tips.groupby('smoker')['tip_pct'].apply(f)

smoker       
No      count    151.000000
        mean       0.159328
        std        0.039910
        min        0.056797
        25%        0.136906
        50%        0.155625
        75%        0.185014
        max        0.291990
Yes     count     93.000000
        mean       0.163196
        std        0.085119
        min        0.035638
        25%        0.106771
        50%        0.153846
        75%        0.195059
        max        0.710345
Name: tip_pct, dtype: float64

In [38]:
#禁止分组键
#分组键会更原始对象的索引共同构成结果对象的层次化索引
tips.groupby('smoker',group_keys=False).apply(top)
#此时分组的column还在columns中，index是原来的index
#

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [39]:
tips.groupby('smoker',as_index=False).apply(top)
#使用上一节的as_index=False似乎不是很合适

total_bill   tip smoker   day    time  size   tip_pct
0 88        24.71  5.85     No  Thur   Lunch     2  0.236746
  185       20.69  5.00     No   Sun  Dinner     5  0.241663
  51        10.29  2.60     No   Sun  Dinner     2  0.252672
  149        7.51  2.00     No  Thur   Lunch     2  0.266312
  232       11.61  3.39     No   Sat  Dinner     2  0.291990
1 109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
  183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
  67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
  178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
  172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [40]:
frame = DataFrame({'data1':np.random.randn(1000),
                   'data2':np.random.randn(1000)})
frame

,data1,data2
0,0.167868,2.124737
1,-0.644444,-2.184861
2,-0.313461,-0.105813
3,0.786305,-0.782414
4,-0.320154,-0.504268
...,...,...
995,-0.420914,-0.506078
996,0.369855,-1.818282
997,-1.095592,-1.904734
998,-1.138003,0.099007


In [41]:
factor = pd.cut(frame.data1,4)
factor
#factor将每个数据分组 总共分为四组
#此时的数据非常适合groupby

0       (0.0179, 1.494]
1      (-1.458, 0.0179]
2      (-1.458, 0.0179]
3       (0.0179, 1.494]
4      (-1.458, 0.0179]
             ...       
995    (-1.458, 0.0179]
996     (0.0179, 1.494]
997    (-1.458, 0.0179]
998    (-1.458, 0.0179]
999    (-1.458, 0.0179]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-2.941, -1.458] < (-1.458, 0.0179] < (0.0179, 1.494] < (1.494, 2.971]]

In [43]:
def get_stats(group):
    return {'min':group.min(),'max':group.max(),
            'count':group.count(),'mean':group.mean()}

grouped = frame['data2'].groupby(factor)
#使用cut之后的Categories对象进行groupby
grouped.size()

data1
(-2.941, -1.458]     84
(-1.458, 0.0179]    427
(0.0179, 1.494]     411
(1.494, 2.971]       78
Name: data2, dtype: int64

In [44]:
grouped.apply(get_stats)

data1                  
(-2.941, -1.458]  min       -3.433580
                  max        3.049054
                  count     84.000000
                  mean       0.111376
(-1.458, 0.0179]  min       -2.588685
                  max        2.678032
                  count    427.000000
                  mean      -0.065280
(0.0179, 1.494]   min       -3.209837
                  max        3.489360
                  count    411.000000
                  mean      -0.039147
(1.494, 2.971]    min       -2.692702
                  max        2.159073
                  count     78.000000
                  mean      -0.134824
Name: data2, dtype: float64

In [45]:
grouped.apply(get_stats).unstack()


,min,max,count,mean
data1,,,,
"(-2.941, -1.458]",-3.433580,3.049054,84.0,0.111376
"(-1.458, 0.0179]",-2.588685,2.678032,427.0,-0.065280
"(0.0179, 1.494]",-3.209837,3.489360,411.0,-0.039147
"(1.494, 2.971]",-2.692702,2.159073,78.0,-0.134824


In [46]:
grouping = pd.qcut(frame['data1'],10,labels=False)
grouping

0      5
1      2
2      3
3      7
4      3
      ..
995    3
996    6
997    1
998    1
999    3
Name: data1, Length: 1000, dtype: int64

In [47]:
grouped2 = frame['data2'].groupby(grouping)
grouped2.size()
#使用qcut将十个组每个组都分成均等的

data1
0    100
1    100
2    100
3    100
4    100
5    100
6    100
7    100
8    100
9    100
Name: data2, dtype: int64

In [48]:
grouped2.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.433580,3.049054,100.0,0.076044
1,-2.534894,1.716296,100.0,-0.106278
2,-2.184861,2.428971,100.0,-0.119777
3,-2.588685,2.678032,100.0,-0.006135
4,-1.899168,2.660682,100.0,0.013804
5,-3.209837,3.489360,100.0,-0.074295
6,-2.359463,2.338645,100.0,0.053853
7,-2.531213,2.300126,100.0,-0.129296
8,-2.466770,2.625823,100.0,-0.045967
